In [221]:
import pandas as pd
import numpy as np
import os
from itertools import combinations
import datacompy
folder_path=r'C:\Users\hoa.nd\Desktop\WAP\compare\report_7'

In [222]:

try:
  df_cognos = pd.read_pickle('df_cognos.pkl')
  df_looker = pd.read_pickle('df_looker.pkl')
except FileNotFoundError:
    # If pickle files are not found, read Excel files
  df_cognos = pd.read_excel(r'C:\Users\hoa.nd\Desktop\WAP\compare\report_7\Cognos.xlsx', dtype=str)
  df_looker = pd.read_excel(r'C:\Users\hoa.nd\Desktop\WAP\compare\report_7\Looker.xlsx', dtype=str)
  
  # Save DataFrames to pickle files
  df_cognos.to_pickle('df_cognos.pkl')
  df_looker.to_pickle('df_looker.pkl')

df_cognos = df_cognos.drop('Report Date*', axis=1)


In [223]:
COGNOS_TO_LOOKER = {
  'Amount in USD*':'Amount in Local Currency - USD for Op Co.',
  'Document type':'Document Type'
}
df_cognos.rename(columns=COGNOS_TO_LOOKER, inplace=True)
print(df_cognos.columns)
print(df_looker.columns)


Index(['General Ledger Account', 'G/L Account: Long Text', 'Region',
       'Contract Number/Doc Hder Txt Office Code', 'Document Type',
       'Posting Year/Month (Calendar)', 'Posting Date in the Document',
       'Amount in Local Currency - USD for Op Co.'],
      dtype='object')
Index(['General Ledger Account', 'G/L Account: Long Text', 'Region',
       'Contract Number/Doc Hder Txt Office Code', 'Document Type',
       'Posting Year/Month (Calendar)', 'Posting Date in the Document',
       'Amount in Local Currency - USD for Op Co.'],
      dtype='object')


In [224]:
REMOVE_COLUMN=[
  # 'General Ledger Account', 
  # 'G/L Account: Long Text', 
  # 'Region',
  # 'Contract Number/Doc Hder Txt Office Code',
  # 'Document Type',
  # 'Posting Year/Month (Calendar)', 
  # 'Posting Date in the Document',
  'Amount in Local Currency - USD for Op Co.'
]

# df_cognos=df_cognos.drop(columns=REMOVE_COLUMN)
# df_looker=df_looker.drop(columns=REMOVE_COLUMN)
df_cognos['Region'] = df_cognos['Region'].replace('No code', 'No office code')

LIST_COLUMN=[
  'General Ledger Account', 
  'G/L Account: Long Text', 
  'Region',
  'Contract Number/Doc Hder Txt Office Code',
  'Document Type',
  'Posting Year/Month (Calendar)', 
  'Posting Date in the Document',
  'Amount in Local Currency - USD for Op Co.'
]
keys_list = [x for x in list(LIST_COLUMN) if x not in REMOVE_COLUMN]
print(keys_list)

# compare = datacompy.Compare( df_cognos, df_looker, join_columns=keys_list, df1_name='Cognos', df2_name='Looker')
# print(compare.report())

['General Ledger Account', 'G/L Account: Long Text', 'Region', 'Contract Number/Doc Hder Txt Office Code', 'Document Type', 'Posting Year/Month (Calendar)', 'Posting Date in the Document']


In [225]:
# Hiển thị danh sách column và số lượng column
def get_column_lists_and_counts(df_cognos, df_looker):
  columns_df_cognos = df_cognos.columns.tolist()
  columns_df_looker = df_looker.columns.tolist()
  num_columns_df_cognos = len(columns_df_cognos)
  num_columns_df_looker = len(columns_df_looker)
  print("Columns in df_cognos:", columns_df_cognos)
  print("Number of columns in df_cognos:", num_columns_df_cognos)
  print("Columns in df_looker:", columns_df_looker)
  print("Number of columns in df_looker:", num_columns_df_looker)
  return columns_df_cognos, num_columns_df_cognos, columns_df_looker, num_columns_df_looker

# Định nghĩa hàm để xóa các cột từ DataFrame
def drop_columns(df, columns_to_drop):
  df = df.drop(columns=columns_to_drop)
  return df

# Hàm Đổi các cột có nhãn (Ví dụ: Y -> Yes, N -> No)
def replace_values(df, columns, replacement):
  for column in columns:
    df[column] = df[column].replace(replacement)
  return df

#Định nghĩa hàm xóa dấu phẩy và convert cột sang kiểu Float để so sánh các cột số
def replace_comma_and_convert_to_float(df, column_names):
  for column_name in column_names:
    if column_name in df.columns:  # Kiểm tra xem cột có tồn tại trong DataFrame không
      if df[column_name].dtype == 'object' and df[column_name].str.contains(',').any():
        df[column_name] = df[column_name].str.replace(',', '', regex=True)
      df[column_name] = df[column_name].astype(float)


# Convert sang datetime
def convert_columns_to_datetime(df, columns):
    for column in columns:
        try:
            df[column] = pd.to_datetime(df[column], errors='coerce')
        except (KeyError, ValueError):
            print(f"Column '{column}' does not exist or cannot be converted to datetime.")

# fillna cho các cột chuỗi và số
def fillna_custom(df, string_columns, numeric_columns):
    for col in string_columns:
        if col in df.columns:
            df[col] = df[col].fillna("No_Data")
    for col in numeric_columns:
        if col in df.columns:
            df[col] = df[col].fillna(0)
    return df

#Định nghĩa hàm trả về kiểu dữ liệu các column của DF
def check_and_print_column_data_types(df, name):
    data_types_df = df.dtypes
    print(f"Data types for DataFrame {name}:")
    print(data_types_df)

# KIỂM TRA CÁC COLUMN ĐÃ CHỌN CÓ TẠO RA ĐƯỢC TỔ HỢP KHÓA HAY KHÔNG
def check_columns_combination_uniqueness(df_cognos, df_looker, columns_to_check):
    def combine_columns(df):
        return df[columns_to_check].apply(lambda x: '-'.join(x.astype(str).str.upper()), axis=1)
    df_cognos_combined = combine_columns(df_cognos)
    df_looker_combined = combine_columns(df_looker)
    are_columns_combined_unique = df_cognos_combined.is_unique and df_looker_combined.is_unique
    if are_columns_combined_unique:
        result = True
    else:
        result = False
    return result

#ĐỊNH NGHĨA HÀM ĐỂ CHUYỂN SANG CHỮ IN HOA
def uppercase_columns(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = df[col].str.upper()
    return df

# ĐỊNH NGHĨA HÀM: TÌM VÀ CẮT DUOID (VD: _cognos, _looker)
def find_and_strip_columns_with_suffix(df, suffix):
    columns_with_suffix = [col for col in df.columns if col.endswith(suffix)]
    stripped_columns = [col.replace(suffix, '') for col in columns_with_suffix]
    return stripped_columns

# ĐỊNH NGHĨA HÀM: TÌM RA SỰ KHÁC BIỆT CỦA 2 DF (MISSING, DIFF) RÒI FILLNA
# Tìm sự khác biẹt
def create_diff_df(df_merged, columns):
    conditions = []
    for column in columns:
        data_cognos = df_merged[f"{column}_cognos"]
        data_looker = df_merged[f"{column}_looker"]
        if data_cognos.dtype== np.float64:
            data_cognos = data_cognos.round(2)
            data_looker = data_looker.round(2)
            condition = np.isclose(data_cognos, data_looker, atol=0.1)
        elif data_cognos.dtype== np.datetime64:
            rounded_cognos = data_cognos.dt.round('S')
            rounded_looker = data_looker.dt.round('S')
            condition = (rounded_cognos == rounded_looker)
        elif data_cognos.dtype== object:
            condition = (data_cognos.str.upper() == data_looker.str.upper())
        else:
            continue
        conditions.append(condition)
    combined_condition = np.all(conditions, axis=0)
    diff_df = df_merged[~combined_condition].copy()
    matching_df = df_merged[combined_condition].copy()
    diff_df = diff_df.applymap(lambda x: 'Not exists' if pd.isnull(x) else x)
    matching_df = matching_df.applymap(lambda x: 'Not exists' if pd.isnull(x) else x)
    return matching_df, diff_df


# LỌC RA PHẦN MISSING HAY DIFF
def filter_and_return_dataframes(df):
    df_missing_cognos = df.loc[df.filter(like='_cognos').eq('Not exists').all(axis=1)]
    df_missing_looker = df.loc[df.filter(like='_looker').eq('Not exists').all(axis=1)]
    df_remaining_diff = df.drop(df_missing_cognos.index).drop(df_missing_looker.index)
    return df_missing_cognos, df_missing_looker, df_remaining_diff.reset_index()

# TÁCH DF CỦA COGNOS VÀ LOOKER
def split_dataframe(df):
    cols_df_cognos = [col for col in df.columns if not col.endswith("_looker")]
    df_cognos_new = df[cols_df_cognos]
    cols_df_looker = [col for col in df.columns if not col.endswith("_cognos")]
    df_looker_new = df[cols_df_looker]
    return df_cognos_new, df_looker_new

# COMPARE VÀ LÊN MÀU CHO SỰ KHÁC BIẸT
def compare_and_style_data(diff_df, keys, suffix_cognos, suffix_looker):
    def highlight_cells(val):
        if val == 'Same':
            return ''
        else:
            return 'background-color: red; color: white;'
    common_prefixes = set(col.split(suffix_cognos)[0] for col in diff_df.columns[2:])
    for prefix in common_prefixes:
        cognos_cols = [col for col in diff_df.columns if col.startswith(f'{prefix}{suffix_cognos}')]
        looker_cols = [col for col in diff_df.columns if col.startswith(f'{prefix}{suffix_looker}')]
        for cognos_col, looker_col in zip(cognos_cols, looker_cols):
            column_name = f'Compare_{prefix}_(Cognos <> Looker)'
            data_type = diff_df[cognos_col].dtype
            if data_type == np.float64:
                diff_df[cognos_col] = diff_df[cognos_col].round(2)
                diff_df[looker_col] = diff_df[looker_col].round(2)
                condition = np.isclose(diff_df[cognos_col], diff_df[looker_col], atol=0.1)
            elif data_type == np.datetime64:
                rounded_cognos = diff_df[cognos_col].dt.round('S')
                rounded_looker = diff_df[looker_col].dt.round('S')
                condition = (rounded_cognos == rounded_looker)
            elif data_type == object:
                condition = (diff_df[cognos_col].str.upper() == diff_df[looker_col].str.upper())
            else:
                continue
            diff_df[column_name] = diff_df.apply(lambda row: f'({row[cognos_col]} <> {row[looker_col]})'
                                                if not condition[row.name]
                                                else 'Same', axis=1)
    selected_keys = keys + [col for col in diff_df.columns if col.startswith('Compare')]
    styled_df = diff_df[selected_keys].style.applymap(highlight_cells, subset=pd.IndexSlice[:, diff_df[selected_keys].columns.str.startswith('Compare_')])
    return styled_df


# ĐỊNH NGHĨA HÀM TÍNH TOÁN
def total_numeric_columns(df_Cognos_cal, df_Looker_cal, columns_to_compare, sales_week_col, start_week=None, end_week=None):
    df_Looker_cal['Source'] = 'Looker'
    df_Cognos_cal['Source'] = 'Cognos'
    combined_data = pd.concat([df_Cognos_cal, df_Looker_cal])
    if start_week and end_week:
        filtered_data = combined_data[(combined_data[sales_week_col] >= start_week) & (combined_data[sales_week_col] <= end_week)]
    else:
        filtered_data = combined_data
    filtered_data[columns_to_compare] = filtered_data[columns_to_compare].round(2)
    result = filtered_data.pivot_table(index=[sales_week_col], columns='Source', values=columns_to_compare, aggfunc='sum')
    for column in columns_to_compare:
        result[f'Compare_{column}_(Cognos <> Looker)'] = result.apply(lambda row: 'Same' if (np.isclose(row[column]['Cognos'], row[column]['Looker'], atol=0.1) == True) else f"{row[column]['Cognos']} <> {row[column]['Looker']}", axis=1)
    result = result.swaplevel(axis=1).sort_index(axis=1, level=[1, 0], ascending=[True, False])
    return result

# ĐỊNH NGHĨA HÀM ĐỂ ĐẶT LẠI TÊN CŨ TRƯỚC KHI XUẤT FILE
# Định nghĩa hàm clean_and_reorder_columns nếu chưa có
def clean_and_reorder_columns(df, suffix_to_remove, new_column_order, new_column_names, output_file):
    df.columns = [col.replace(suffix_to_remove, '') for col in df.columns]
    df = df[new_column_order]
    df.rename(columns=dict(zip(df.columns, new_column_names)), inplace=True)
    df.to_csv(os.path.join(folder_path, output_file), index=False)
    return df

In [226]:
df_merged = pd.merge(df_cognos, df_looker,  on=keys_list, how='outer', suffixes=('_cognos', '_looker'))
matching_df, diff_df = create_diff_df(df_merged, REMOVE_COLUMN)


In [227]:
df_missing_cognos, df_missing_looker, df_remaining_diff = filter_and_return_dataframes(diff_df)
df_cognos_diff_new, df_looker_diff_new = split_dataframe(df_remaining_diff)
df_not_in_cognos, df_only_in_looker = split_dataframe(df_missing_cognos)
df_only_in_cognos, df_not_in_looker = split_dataframe(df_missing_looker)

df_missing_cognos.to_excel(os.path.join(r'C:\Users\hoa.nd\Desktop\WAP\compare\report_7', 'df_missing_cognos.xlsx'), index=False)
df_missing_looker.to_excel(os.path.join(r'C:\Users\hoa.nd\Desktop\WAP\compare\report_7', 'df_missing_looker.xlsx'), index=False)
